# LAB 2: Recommender System from Scratch

Giorgio Lazzarinetti - My Contacts
For any questions or doubts you can find my contacts here:

giorgiolazzarinetti@gmail.com g.lazzarinetti@campus.unimib.it

## Notebook Outline

- **Introduction to Recommender System**
- **Movielens Dataset**
- **Generalized Matrix Factorization Model**
- **Neural Collaborative Filtering**
- **LAB CHALLENGE 1: Neural Matrix Factorization**

## References

The architecture of the deep model, the evaluation strategy and the metrics used are taken from the paper: ["Neural Collaborative Filtering"](https://arxiv.org/abs/1708.05031) by He Xiangnan, Liao Lizi, Zhang Hanwang, Nie Liqiang, Hu Xia and Chua Tat-Seng - In Porc. of the 26th Interantional Conference on World Wide Web - 2017.

## Introduction to Recommender System

Recommender systems are algorithms that mimic the psychology and personality of humans, in order to predict their needs and desires. More formally, recommender systems adopt data-mining and machine-learning techniques to help users in finding attractive and useful products. Products can be almost anything: physical items (e.g., smartphones), places (e.g., restaurants), digital content (e.g., movies and music), and many more. Recommender systems produce recommendations based on different inputs: demographic information about users, ratings and comments on products, individual’s or community’s past preferences and choices, social networks, context of use.

There are many different types of techniques and implementations out there.

- **Content-based methods** uses attributes of items to recommend to users new items similar to what the user has liked in the past (doesn't take into account the behaviour of other users);
- **Collaborative Filtering methods** uses similarities between users and items simultaneously to determine recommendations;
- **Hybrid methos** mix content-based and collaborative filtering approaches.

Other approaches are also called **Knowlege-based methods** which uses explicit knowledge about users and items to build recommendations criteria with a rule-based approach.


<center>  <img src="https://drive.google.com/uc?export=view&id=1Qaizz9YLvqgXg0blWFwN92IQSQPLTSoH" width="950" height="400"> </center>


In the following we'll focus on Collaborative Filtering methods, with a model-based approach with deep learning algorithms.

### Problem Definition

Given a past record of movies seen by a user, we will build a recommender system that helps the user discover movies of their interest.

Specifically, given <userID, itemID> occurrence pairs, we need to generate a ranked list of movies for each user.

We model the problem as a binary classification problem, where we learn a function to predict whether a particular user will like a particular movie or not.

$$f(userid, itemid) →, [0,1]$$

The model takes in two sparse vectors, one representing the user and the other represents items. The users vector has size #users, while the items vector has size #items.  

So, elaborately,
- User vector=[0,0,1...,0,0,0] with m elements, means this vector represents the 3 rd user out of m.
- Item vector=[0,1,0,0,0,0...0] with n elements, means this vector represents the 2 nd item out of n.

Basically both items and users are one-hot encoded.

These two vectors should be passed to a first embedding layer  (to project sparse representations to dense ones). These embeddings can be seen as a latent vector for users and items.

Thus, the final predictive model will be
$$y_{ui} = f(\mathbf{P}^T\mathbf{v}_u^T, \mathbf{Q}^T\mathbf{v}_i^T | \mathbf{P}, \mathbf{Q}, \mathbf{\Theta}_f) $$

where **P** and **Q** denotes the latent factor matrix for users and items and **$\Theta_f$** denotes the model parameters.


### Packages

In [5]:
import os
import time
import random
import argparse
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader

In [6]:
np.random.seed(7)
torch.manual_seed(0)

In [7]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

## Movielens Dataset

### Dataset

We use the MovieLens 100K dataset, which has 100,000 ratings from 1000 users on 1700 movies.

The ratings are given to us in form of <userID,itemID, rating, timestamp> tuples. Each user has a minimum of 20 ratings.

You can download the dataset [here](https://grouplens.org/datasets/movielens/). Download the file ml-100k.zip. Unzip it and extract the file u.data

### Create Dataset

Here we are going to create the necessary dataset for building recommender system.

After downloading the file as indicated, save them in your Colab Notebooks directory. In order to let the Notebook see the file on your Drive you have to mount it.

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [147]:
dataset_origin = {'100k': 'u.data', '1M': 'ratings.dat'}
#if I want to use the bigger dataset I just write change the numsample data
num_sample_data = '100k'

#DATA_PATH = 'drive/MyDrive/Colab Notebooks/2 - Data/{}'.format(dataset_origin[num_sample_data]) #change this with your directory
#MODEL_PATH = 'drive/MyDrive/Colab Notebooks/2 - Models/movielens_{}/'.format(num_sample_data) #change this with your directory
DATA_PATH = '/content/drive/MyDrive/Uni/1st year/2nd semester/AI, Machine Learning/2  - Recommender System from scratch [LC] (1).ipynb/ml-100k/ml-100k/u.data'.format(dataset_origin[num_sample_data]) #change this with your directory
MODEL_PATH = '/content/drive/MyDrive/Uni/1st year/2nd semester/AI, Machine Learning/2  - Recommender System from scratch [LC] (1).ipynb/Model/movielens_{}/'.format(num_sample_data) #change this with your directory

In [148]:
GENRE_PATH = '/content/drive/MyDrive/Uni/1st year/2nd semester/AI, Machine Learning/2  - Recommender System from scratch [LC] (1).ipynb/ml-100k/ml-100k/u.genre'.format(dataset_origin[num_sample_data]) #change this with your director
INFO_PATH = '/content/drive/MyDrive/Uni/1st year/2nd semester/AI, Machine Learning/2  - Recommender System from scratch [LC] (1).ipynb/ml-100k/ml-100k/u.info'.format(dataset_origin[num_sample_data]) #change this with your director
ITEM_PATH = '/content/drive/MyDrive/Uni/1st year/2nd semester/AI, Machine Learning/2  - Recommender System from scratch [LC] (1).ipynb/ml-100k/ml-100k/u.item'.format(dataset_origin[num_sample_data]) #change this with your director
OCCUPATION_PATH = '/content/drive/MyDrive/Uni/1st year/2nd semester/AI, Machine Learning/2  - Recommender System from scratch [LC] (1).ipynb/ml-100k/ml-100k/u.occupation'.format(dataset_origin[num_sample_data]) #change this with your director
USER_PATH = '/content/drive/MyDrive/Uni/1st year/2nd semester/AI, Machine Learning/2  - Recommender System from scratch [LC] (1).ipynb/ml-100k/ml-100k/u.user'.format(dataset_origin[num_sample_data]) #change this with your director

We now build some function to manage the data.

We drop the exact value of rating (1,2,3,4,5) and instead convert it to an implicit scenario i.e. any positive interaction is given value of 1. All other interactions are given a value of zero, by default.

Since we are training a classifier, we need both positive and negative samples. The records present in the dataset are counted as positive samples. We assume that all entries in the user-item interaction matrix are negative samples (a strong assumption, and easy to implement).

We randomly sample 4 items that are not interacted by the user, for every item interacted by the user. This way, if a user has 20 positive interactions, he will have 80 negative interactions. These negative interactions cannot contain any positive interaction by the user, though they may not be all unique due to random sampling.

In [163]:
### WHOLE DATA ###
genre_data = pd.read_csv(GENRE_PATH, sep = '|', names = ["genre", "code"])
info_data = pd.read_csv(INFO_PATH, names =["Info"])
occupation_data = pd.read_csv(OCCUPATION_PATH, names = ["occupations"])
data_data = pd.read_csv(DATA_PATH, sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])
user_data = pd.read_csv(USER_PATH, sep='|', names=['user_id', 'age', 'gender', 'occupation', 'zip_code'])

column_names = ['movie id', 'movie title', 'release date', 'video release date',
                'IMDb URL', 'unknown', 'Action', 'Adventure', 'Animation',
                "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
                'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
                'Thriller', 'War', 'Western']
item_data = pd.read_csv(ITEM_PATH, sep='|', encoding="latin-1", header=None, names=column_names, engine='python')
## NOTES ON ITEM DATA ##
# a 1 indicates the movie is of that genre, a 0 indicates it is not;
# movies can be in several genres at once.
# The movie ids are the ones used in the u.data data set.

In [ ]:
genre_data.head()
#genre_data.isnull().any()

,genre,code
0,unknown,0
1,Action,1
2,Adventure,2
3,Animation,3
4,Children's,4


In [ ]:
info_data.head()

,Info
0,943 users
1,1682 items
2,100000 ratings


In [ ]:
occupation_data.head()
#occupation_data.isnull().any()

,occupations
0,administrator
1,artist
2,doctor
3,educator
4,engineer


In [156]:
item_data.head()
#item_data.isnull().any()
#release date and video release date have some null values, but we can drop those columns bc we dont care about them

,item_id,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,FilmNoir,Horror,Musical,Mystery,Romance,SciFi,Thriller,War,Western
0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [158]:
data_data.head()
#data_data.isnull().any()

,user_id,item_id,rating,timestamp,age,gender_F,gender_M,occupation_administrator,occupation_artist,occupation_doctor,...,FilmNoir,Horror,Musical,Mystery,Romance,SciFi,Thriller,War,Western,rank_latest
0,0,0,1.0,881250949,49,False,True,False,False,False,...,0,0,0,0,0,0,0,0,0,37.0
1,1,1,1.0,891717742,39,True,False,False,False,False,...,1,0,0,1,0,0,1,0,0,19.0
2,2,2,1.0,878887116,25,False,True,False,False,False,...,0,0,0,0,0,0,0,0,0,76.0
3,3,3,1.0,880606923,28,False,True,False,False,False,...,0,0,0,0,1,0,0,1,1,61.0
4,4,4,1.0,886397596,47,False,True,False,False,False,...,0,0,0,0,0,0,0,0,0,13.0


In [ ]:
user_data.head()
#user_data['age'].isnull().any()

,user_id,age,zip_code,gender_F,gender_M,occupation_administrator,occupation_artist,occupation_doctor,occupation_educator,occupation_engineer,...,occupation_marketing,occupation_none,occupation_other,occupation_programmer,occupation_retired,occupation_salesman,occupation_scientist,occupation_student,occupation_technician,occupation_writer
0,1,24,85711,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,2,53,94043,True,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2,3,23,32067,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3,4,24,43537,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
4,5,33,15213,True,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False


In [164]:
user_data = pd.get_dummies(user_data, columns=['gender', 'occupation'])
user_data.head()
user_data.drop(columns = ["zip_code"], inplace = True)
# movie title and imdb url are not useful. Release date and video rel date have missing values but are not useful
columns_to_drop = ['movie title', 'IMDb URL', 'release date', 'video release date']
item_data.drop(columns=columns_to_drop, inplace=True)

In [165]:
item_data.rename(columns={'movie id': 'item_id'}, inplace=True)
item_data.rename(columns={"Children's": 'Children'}, inplace=True)
item_data.rename(columns={'Film-Noir': 'FilmNoir'}, inplace=True)
item_data.rename(columns={'Sci-Fi': 'SciFi'}, inplace=True)

In [166]:
# Merge user_data with data_data based on the user_id
data_data = pd.merge(data_data, user_data, on='user_id', how='left')
data_data = pd.merge(data_data, item_data, on='item_id', how='left')

In [142]:
print(list(data_data.columns))

['user_id', 'item_id', 'rating', 'timestamp', 'age', 'gender_F', 'gender_M', 'occupation_administrator', 'occupation_artist', 'occupation_doctor', 'occupation_educator', 'occupation_engineer', 'occupation_entertainment', 'occupation_executive', 'occupation_healthcare', 'occupation_homemaker', 'occupation_lawyer', 'occupation_librarian', 'occupation_marketing', 'occupation_none', 'occupation_other', 'occupation_programmer', 'occupation_retired', 'occupation_salesman', 'occupation_scientist', 'occupation_student', 'occupation_technician', 'occupation_writer', 'unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']


In [ ]:
data_data.head()

,user_id,item_id,rating,timestamp,age,zip_code,gender_F,gender_M,occupation_administrator,occupation_artist,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,196,242,3,881250949,49,55105,False,True,False,False,...,0,0,0,0,0,0,0,0,0,0
1,186,302,3,891717742,39,00000,True,False,False,False,...,0,1,0,0,1,0,0,1,0,0
2,22,377,1,878887116,25,40206,False,True,False,False,...,0,0,0,0,0,0,0,0,0,0
3,244,51,2,880606923,28,80525,False,True,False,False,...,0,0,0,0,0,1,0,0,1,1
4,166,346,1,886397596,47,55113,False,True,False,False,...,0,0,0,0,0,0,0,0,0,0


We now define the class MovieLens Dataset which will be used do read the data and create the train and test dataset.

In [15]:
class Rating_Datset(Dataset):
	def __init__(self, user_list, item_list, rating_list):
		super(Rating_Datset, self).__init__()
		self.user_list = user_list
		self.item_list = item_list
		self.rating_list = rating_list

	def __len__(self):
		return len(self.user_list)

	def __getitem__(self, idx):
		user = self.user_list[idx]
		item = self.item_list[idx]
		rating = self.rating_list[idx]

		return (
			torch.tensor(user, dtype=torch.long),
			torch.tensor(item, dtype=torch.long),
			torch.tensor(rating, dtype=torch.float)
			)

In [203]:
class Rating_Dataset2(Dataset):
	def __init__(self, user_list, item_list, rating_list, demographic_list, genre_list):
		super(Rating_Dataset2, self).__init__()
		self.user_list = user_list
		self.item_list = item_list
		self.rating_list = rating_list
		self.demographic_list = demographic_list
		self.genre_list = genre_list

	def __len__(self):
		return len(self.user_list)

	def __getitem__(self, idx):
		user = self.user_list[idx]
		item = self.item_list[idx]
		rating = self.rating_list[idx]
		demographic = self.demographic_list[idx]
		genre = self.genre_list[idx]

		return (
            torch.tensor(user, dtype=torch.long),
            torch.tensor(item, dtype=torch.long),
            torch.tensor(rating, dtype=torch.float),
            torch.tensor(demographic, dtype=torch.float),
            torch.tensor(genre, dtype=torch.float)
        )

In [211]:
class NCF_Data2(object):
	"""
	Construct Dataset for NCF
	Let's think as ratings as complete data
	"""
	def __init__(self, args, ratings):
		self.ratings = ratings
		self.num_ng = args.num_ng
		self.num_ng_test = args.num_ng_test
		self.batch_size = args.batch_size

		self.preprocess_ratings = self._reindex(self.ratings)

		self.user_pool = set(self.ratings['user_id'].unique())
		self.item_pool = set(self.ratings['item_id'].unique())

		self.train_ratings, self.test_ratings = self._leave_one_out(self.preprocess_ratings)
		self.negatives = self._negative_sampling(self.preprocess_ratings)


	def _reindex(self, ratings):
		"""
		Process dataset to reindex userID and itemID, also set rating as binary feedback
		"""

		user_list = list(ratings['user_id'].drop_duplicates())
		user2id = {w: i for i, w in enumerate(user_list)}

		item_list = list(ratings['item_id'].drop_duplicates())
		item2id = {w: i for i, w in enumerate(item_list)}

		ratings['user_id'] = ratings['user_id'].apply(lambda x: user2id[x])
		ratings['item_id'] = ratings['item_id'].apply(lambda x: item2id[x])
		ratings['rating'] = ratings['rating'].apply(lambda x: float(x > 0))
		return ratings

	def _leave_one_out(self, ratings):
      # Leave-one-out evaluation protocol
		ratings['rank_latest'] = ratings.groupby(['user_id'])['timestamp'].rank(method='first', ascending=False)
		test = ratings.loc[ratings['rank_latest'] == 1]
		train = ratings.loc[ratings['rank_latest'] > 1]
		assert train['user_id'].nunique() == test['user_id'].nunique(), 'Not Match Train User with Test User'
		return train[['user_id', 'item_id', 'rating', 'age', 'gender_F','gender_M', 'occupation_administrator', 'occupation_artist', 'occupation_doctor', 'occupation_educator', 'occupation_engineer', 'occupation_entertainment', 'occupation_executive', 'occupation_healthcare', 'occupation_homemaker', 'occupation_lawyer', 'occupation_librarian', 'occupation_marketing', 'occupation_none', 'occupation_other', 'occupation_programmer', 'occupation_retired', 'occupation_salesman', 'occupation_scientist', 'occupation_student', 'occupation_technician', 'occupation_writer', 'unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'FilmNoir', 'Horror', 'Musical', 'Mystery', 'Romance', 'SciFi', 'Thriller', 'War', 'Western', 'rank_latest']], \
               test[['user_id', 'item_id', 'rating', 'age', 'gender_F', 'gender_M', 'occupation_administrator', 'occupation_artist', 'occupation_doctor', 'occupation_educator', 'occupation_engineer', 'occupation_entertainment', 'occupation_executive', 'occupation_healthcare', 'occupation_homemaker', 'occupation_lawyer', 'occupation_librarian', 'occupation_marketing', 'occupation_none', 'occupation_other', 'occupation_programmer', 'occupation_retired', 'occupation_salesman', 'occupation_scientist', 'occupation_student', 'occupation_technician', 'occupation_writer', 'unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'FilmNoir', 'Horror', 'Musical', 'Mystery', 'Romance', 'SciFi', 'Thriller', 'War', 'Western', 'rank_latest']]

	def _negative_sampling(self, ratings):

		interact_status = (
			ratings.groupby('user_id')['item_id']
			.apply(set)
			.reset_index()
			.rename(columns={'item_id': 'interacted_items'}))
		interact_status['negative_items'] = interact_status['interacted_items'].apply(lambda x: self.item_pool - x)
		interact_status['negative_samples'] = interact_status['negative_items'].apply(lambda x: random.sample(x, self.num_ng_test))
		return interact_status[['user_id', 'negative_items', 'negative_samples']]

	def get_train_instance(self):
		users, items, ratings, demographics, genre = [], [], [], [], []
		train_ratings = pd.merge(self.train_ratings, self.negatives[['user_id', 'negative_items']], on='user_id')
		train_ratings['negatives'] = train_ratings['negative_items'].apply(lambda x: random.sample(x, self.num_ng))

		print(list(train_ratings.columns))
		#demographic_list_train = train_ratings[['age', 'gender_F', 'gender_M', 'occupation_administrator', 'occupation_artist', 'occupation_doctor', 'occupation_educator', 'occupation_engineer', 'occupation_entertainment', 'occupation_executive', 'occupation_healthcare', 'occupation_homemaker', 'occupation_lawyer', 'occupation_librarian', 'occupation_marketing', 'occupation_none', 'occupation_other', 'occupation_programmer', 'occupation_retired', 'occupation_salesman', 'occupation_scientist', 'occupation_student', 'occupation_technician', 'occupation_writer', 'unknown']].values.tolist()
		#genre_list_train = train_ratings[['Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']].values.tolist()
		demographic_list_train =['age', 'gender_F', 'gender_M', 'occupation_administrator', 'occupation_artist', 'occupation_doctor', 'occupation_educator', 'occupation_engineer', 'occupation_entertainment', 'occupation_executive', 'occupation_healthcare', 'occupation_homemaker', 'occupation_lawyer', 'occupation_librarian', 'occupation_marketing', 'occupation_none', 'occupation_other', 'occupation_programmer', 'occupation_retired', 'occupation_salesman', 'occupation_scientist', 'occupation_student', 'occupation_technician', 'occupation_writer', 'unknown']
		genre_list_train = ['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'FilmNoir', 'Horror', 'Musical', 'Mystery', 'Romance', 'SciFi', 'Thriller', 'War', 'Western']

        # Include demographic or genre information in training instances
		for row in train_ratings.itertuples():
			print("ora rows:")
			print(list(row))
			user_id = int(row.user_id)
			item_id = int(row.item_id)
			rating = float(row.rating)
			users.append(user_id)
			items.append(item_id)
			ratings.append(rating)
			for column in demographic_list_train:
				demographics.append(int(row.__getattribute__(column)))
			for column in genre_list_train:
				genre.append(int(row.__getattribute__(column)))


			#demographics.append(demographic_list_train[row.Index])
			#genre.append(genre_list_train[row.Index])
			#for demo in demographic_list_train:
			#	demographics.append(row.'{demo}')
			#for gen in genre_list_train:
			#	genre.append(row.genre)

			for i in range(self.num_ng):
				neg_item_id = int(row.negatives[i])
				users.append(user_id)
				items.append(neg_item_id)
				ratings.append(float(0))
				#should i put them also here???
			#for column in demographic_list_train:
			#	demographics.append(int(row.__getattribute__(column)))
			#for column in genre_list_train:
			#	genre.append(int(row.__getattribute__(column)))


			dataset = Rating_Dataset2(
            user_list=users,
            item_list=items,
            rating_list=ratings,
            demographic_list= demographics,  # Pass demographic information if available
            genre_list=genre  # Pass genre information if available
        )

			return DataLoader(dataset, batch_size=self.batch_size, shuffle=True, num_workers=4)

	def get_test_instance(self):
		users, items, ratings, demographics, genre = [], [], [], [], []
		test_ratings = pd.merge(self.test_ratings, self.negatives[['user_id', 'negative_samples']], on='user_id')
		demographic_list_test =['age', 'gender_F', 'gender_M', 'occupation_administrator', 'occupation_artist', 'occupation_doctor', 'occupation_educator', 'occupation_engineer', 'occupation_entertainment', 'occupation_executive', 'occupation_healthcare', 'occupation_homemaker', 'occupation_lawyer', 'occupation_librarian', 'occupation_marketing', 'occupation_none', 'occupation_other', 'occupation_programmer', 'occupation_retired', 'occupation_salesman', 'occupation_scientist', 'occupation_student', 'occupation_technician', 'occupation_writer', 'unknown']
		genre_list_test = ['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'FilmNoir', 'Horror', 'Musical', 'Mystery', 'Romance', 'SciFi', 'Thriller', 'War', 'Western']
	      # Include demographic or genre information in test instances
		for row in test_ratings.itertuples():
			user_id = int(row.user_id)
			item_id = int(row.item_id)
			rating = float(row.rating)
			users.append(user_id)
			items.append(item_id)
			ratings.append(rating)
			for column in demographic_list_test:
				demographics.append(int(row.__getattribute__(column)))
			for column in genre_list_test:
				genre.append(int(row.__getattribute__(column)))

			for neg_item_id in getattr(row, 'negative_samples'):
				users.append(user_id)
				items.append(int(neg_item_id))
				ratings.append(float(0))
			#for column in demographic_list_test:
			#	demographics.append(int(row.__getattribute__(column)))
			#for column in genre_list_test:
			#	genre.append(int(row.__getattribute__(column)))

			dataset = Rating_Dataset2(
            user_list=users,
            item_list=items,
            rating_list=ratings,
            demographic_list=demographics,  # Pass demographic information if available
            genre_list=genre  # Pass genre information if available
        )
			#print("users:", users)
			print("train load demo: ", demographics)
			print("train load genre: ", genre)


			return DataLoader(dataset, batch_size=self.num_ng_test + 1, shuffle=False, num_workers=2)

### Evaluation Metrics

We randomly sample 100 items that are not interacted by the user, ranking the test item among the 100 items. We truncate the ranked list at 10.

Since it is too time-consuming to rank all items for every user, for we will have to calculate 1000\*1700 ~10⁶ values. With this strategy, we need 1000*100 ~ 10⁵ values, an order of magnitude less.

For each user, we use the latest rating(according to timestamp) in the test set, and we use the rest for training. This evaluation methodology is also known as leave-one-out strategy.

#### Metrics

We use **Hit Ratio** (HR), and **Normalized Discounted Cumulative Gain** (NDCG) to evaluate the performance for our RS.


In recommender settings, the **HR** is simply the fraction of users for which the correct answer is included in the recommendation list of length L.

$$HR = \frac{|U_{hit}^{L}|}{U_{all}}$$

Where $U_{hit}^{L}$ is the number of users for which the correct answer is included in the top L recommendation list, while, $U_{all}$ is the total number of user in the test dataset. Clearly the larger L is the larger HR become.

**NDCG** is a measure of ranking quality. In information retrieval, it is often used to measure effectiveness of web search engine algorithms or related applications. Using a graded relevance scale of documents in a search-engine result set, DCG measures the usefulness, or gain, of a document based on its position in the result list. The gain is accumulated from the top of the result list to the bottom, with the gain of each result discounted at lower ranks

**Gain** for an item is essentialy the same as the relevance score, which can be numerical ratings like search results in Google which can be rated in scale from 1 to 5, or binary in case of implicit data where we only know if a user has consumed certain item or not. Naturally **Cumulative Gain** is defined as the sum of gains up to a position k in the recommendation list.

$$CG(k) = \sum_{i=1}^{k}G_i$$

One obvious drawback of CG is that it does not take into account of ordering. By swapping the relative order of any two items, the CG would be unaffected. This is problematic when ranking order is important. For example, on Google Search results, you would obviously not like placing the most relevant web page at the bottom.

To penalize highly relevant items being placed at the bottom, we introduce the **Discounted Cumulative Gaing** (DCG).

$$DCG(k) = \sum_{i=1}^{k} \frac{G_i}{log_2(i+1)}$$

By diving the gain by its rank, we sort of push the algorithm to place highly relevant items to the top to achieve the best DCG score.

There is still a drawback of DCG score. It is that DCG score adds up with the length of recommendation list. Therefore, we cannot consistently compare the DCG score for system recommending top 5 and top 10 items, because the latter will have higher score not because its recommendation quality but pure length.

We tackle this issue by introducing **Ideal Discounted Cumulative Gain** (IDCG). IDCG is the DCG score for the most ideal ranking, which is ranking the items top down according their relevance up to position k.

$$IDCG(k) = \sum_{i=1}^{|I(k)|} \frac{G_i}{log_2(i+1)}$$

Where $|I(k)|$ represent the ideal list of items up to position k.

And NDCG is simply to normalize the DCG score by IDCG such that its value is always between 0 and 1 regardless of the length.

$$NDCD(k) = \frac{DCG(k)}{IDCG(k)}$$

Our model gives a confidence score between 0 and 1 for each item present in the test set for a given user. The items are sorted in decreasing order of their score, and top 10 items are given as recommendation. If the test item (which is only one for each user) is present in this list, HR is one for this user, else it is zero. The final HR is reported after averaging for all users. A similar calculation is done for NDCG.

While training, we will be minimizing the cross-entropy loss, which is the standard loss function for a classification problem. The real strength of RS lies in giving a ranked list of top-k items, which a user is most likely to interact. Think about why you mostly click on google search results only on the first page, and never go to other pages. Metrics like NDCG and HR help in capturing this phenomenon by indicating the quality of our ranked lists.

In [18]:
def hit(ng_item, pred_items):
	if ng_item in pred_items:
		return 1
	return 0


def ndcg(ng_item, pred_items):
	if ng_item in pred_items:
		index = pred_items.index(ng_item)
		return np.reciprocal(np.log2(index+2))
	return 0


def metrics(model, test_loader, top_k, device):
	HR, NDCG = [], []

	for user, item, label in test_loader:
		user = user.to(device)
		item = item.to(device)

		predictions = model(user, item)
		_, indices = torch.topk(predictions, top_k)
		recommends = torch.take(
				item, indices).cpu().numpy().tolist()

		ng_item = item[0].item() # leave one-out evaluation has only one item per user
		HR.append(hit(ng_item, recommends))
		NDCG.append(ndcg(ng_item, recommends))

	return np.mean(HR), np.mean(NDCG)

In [209]:
def metrics2(model, test_loader, top_k, device):
    HR, NDCG = [], []

    for user, item, demographic, genre, label in test_loader:
        user = user.to(device)
        item = item.to(device)
        demographic = demographic.to(device)
        genre = genre.to(device)
        demographic = demographic.long()
        genre = genre.long()

        predictions = model(user, item, demographic, genre)
        _, indices = torch.topk(predictions, top_k)
        recommends = torch.take(item, indices).cpu().numpy().tolist()

        ng_item = item[0].item()  # Leave one-out evaluation has only one item per user
        HR.append(hit(ng_item, recommends))
        NDCG.append(ndcg(ng_item, recommends))

    return np.mean(HR), np.mean(NDCG)

## Generalized Matrix Factorization (GMF)

Generally Matrix Factorization (MF) algorithms associates each user and item with a real-valued vector of latent features.

Let $\mathbf{p_u}$ and $\mathbf{q_i}$ denote the latent vector for user u and item i, respectively; MF estimates an interaction y_{ui} as the inner product of $\mathbf{p_u}$ and $\mathbf{q_i}$:

$$y_{ui} = f(u, i| \mathbf{p_u}, \mathbf{q_i}) = \mathbf{p_u}^T\mathbf{q_i} = \sum_{k=1}^Kp_{uk}q_{ik} $$

Where K denotes the dimension of the latent space.

MF models the dimension of the interaction of user and item latent factors, assuming each dimension of the latent space is independent of each other and linearly combining them  with the same weights.

This imposes some limitation of MF caused by the use of a simple and fixed inner product to estimate complex user-item interactions in the low-dimensional latent space.

In order to overcome this limitation, we can build a Generalized Matrix Factorization (GMF) algorithm where we can weight the linear combination of the element-wise product and use this with an activation function to learn a representation of the input insted of using a fixed one.

Let the user latent vector $\mathbf{p_u}$  be denoted as $\mathbf{P}^T\mathbf{v}_u^T$ and the item latent vector $\mathbf{q_i}$ as $\mathbf{Q}^T\mathbf{v}_i^T$.

The GMF can be expressed as
$$Y_{ui} = a_{out}(\mathbf{h}^T(\mathbf{p_u}\odot \mathbf{q_i})$$
where $a_{out}$ is the activation function and $\mathbf{h}$ are the edge weights of the output layer.

Intuitively, if we use an identity function as $a_{out}$ and enforce $\mathbf{h}$ to be a uniform vector of 1, we can exactly recover the MF model.

In the following we will use $\mathbf{h}$ as linear layer and a $a_{out}$ as sigmoid function to learn $\mathbf{h}$ weigths from data with the log loss.



In [19]:
class GMF(nn.Module):
    def __init__(self, args, num_users, num_items):
        super(GMF, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.factor_num = args.factor_num

        self.embedding_user = nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.factor_num)
        self.embedding_item = nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.factor_num)

        self.affine_output = nn.Linear(in_features=self.factor_num, out_features=1)
        self.logistic = nn.Sigmoid()

    def forward(self, user_indices, item_indices):
        user_embedding = self.embedding_user(user_indices)
        item_embedding = self.embedding_item(item_indices)
        element_product = torch.mul(user_embedding, item_embedding)
        logits = self.affine_output(element_product)
        rating = self.logistic(logits)
        return rating.squeeze()

    def init_weight(self):
        pass

## Neural Collaborative Filtering
GMF learn user and item embedding separately. To empower the model it is possible to combine the features of two pathways by concatenating them and passing this concatenation to a Multi-Layer Perceptron (MLP) to learn the interaction between user and item latent features. This model is known as Neural Collaborative Filtering (NCF)

The input to the model is userID and itemID, which is fed into an embedding layer. Thus, each user and item is given an embedding. Then there are multiple dense layers afterward, followed by a single neuron with a sigmoid activation.

The output of the sigmoid neuron can be interpreted as the probability the user is likely to interact with an item.

The model, we are going to implement is the following:
<center>  <img src="https://drive.google.com/uc?export=view&id=1rL_8kkHIhSlQjWr8hNal4Tyog87-2kNP" width="550" height="350"> </center>

The user in item vectors are passed to an embedding layer that build a dense or latent vectors for the sparse inputs, from the input layer. The obtained latent vectors are fed into the multi-layer neural architecture, to map the latent vectors to the predicted probability scores. The layers are responsible to find the complex user-item relations from the data.

The output layer produces the predicted score $y_(ui)$, i.e, how much is the probability that the user u will interact with the item i.

A pointwise loss function is used to minimize the difference between the target value Y(ui) and the corresponding predicted value.

Formally the model we are going to implement is the following:

$$\mathbf{z_1} = \phi_1 (\mathbf{p_u}, \mathbf{q_i}) = \begin{bmatrix}
\mathbf{p_u} \\ \mathbf{q_i}
\end{bmatrix}$$

$$\phi_L(\mathbf{z}_{L-1}) = a_L(\mathbf{W}_L^T\mathbf{z}_{L-1} + \mathbf{b}_L)$$

$$ y_{ui} = \sigma(\mathbf{h}^T\phi_L(\mathbf{z}_{L-1}))$$

where $\mathbf{W}_x, \mathbf{b}_x, a_x$ denotes the weight matrix, bias vector and activation function for the x-th layer's perceptron


In [20]:
class MLP(nn.Module):
    def __init__(self, args, num_users, num_items):
        super(MLP, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.factor_num = args.factor_num
        self.layers = args.layers

        self.embedding_user = nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.factor_num)
        self.embedding_item = nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.factor_num)

        self.fc_layers = nn.ModuleList()
        for idx, (in_size, out_size) in enumerate(zip(self.layers[:-1], self.layers[1:])):
            self.fc_layers.append(nn.Linear(in_size, out_size))

        self.affine_output = nn.Linear(in_features=self.layers[-1], out_features=1)
        self.logistic = nn.Sigmoid()

    def forward(self, user_indices, item_indices):
        user_embedding = self.embedding_user(user_indices)
        item_embedding = self.embedding_item(item_indices)
        vector = torch.cat([user_embedding, item_embedding], dim=-1)  # the concat latent vector
        for idx, _ in enumerate(range(len(self.fc_layers))):
            vector = self.fc_layers[idx](vector)
            vector = nn.ReLU()(vector)
            # vector = nn.BatchNorm1d()(vector)
            # vector = nn.Dropout(p=0.5)(vector)
        logits = self.affine_output(vector)
        rating = self.logistic(logits)
        return rating.squeeze()

    def init_weight(self):
        pass

In [193]:
import torch.nn.functional as F

class MLP2(nn.Module):
    def __init__(self, args, num_users, num_items, num_demographic_features, num_genre_features):
        super(MLP2, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.num_demographic_features = num_demographic_features
        self.num_genre_features = num_genre_features
        self.factor_num = args.factor_num
        self.layers = args.layers
        self.num_layers = len(self.layers)

        # Define embeddings for users, items, demographic, and genre features
        self.embedding_user = nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.factor_num)
        self.embedding_item = nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.factor_num)
        self.embedding_demographic = nn.Embedding(num_embeddings=self.num_demographic_features, embedding_dim=self.factor_num)
        self.embedding_genre = nn.Embedding(num_embeddings=self.num_genre_features, embedding_dim=self.factor_num)

        # Define fully connected layers
       # input_size = self.factor_num * 2 + self.factor_num * 2  # User/item embeddings + demographic/genre embeddings
        self.fc_layers = nn.ModuleList()
        for idx in range(self.num_layers):
            if idx == 0:

              input_size = self.factor_num * 2 + self.factor_num * 2  # Initial input size

            else:

              input_size = self.layers[idx - 1]  # Adjust input size based on previous layer's output size
            self.fc_layers.append(nn.Linear(input_size, self.layers[idx]))

        # Output layer
        self.affine_output = nn.Linear(in_features=self.layers[-1], out_features=1)
        self.logistic = nn.Sigmoid()

    def forward(self, user_indices, item_indices, demographic_features, genre_features):
        # Embeddings for users, items, demographic, and genre features
        #print("users idxs: ", user_indices)
        #print("demo features: ", demographic_features)
        user_embedding = self.embedding_user(user_indices)
        print("users ok")
        item_embedding = self.embedding_item(item_indices)
        print("items ok")
        demographic_embedding = self.embedding_demographic(demographic_features)
        print("demo ok")
        #print("genres: ", genre_features)
        print("inside genre", genre_features)
        genre_embedding = self.embedding_genre(genre_features)
        print("genres ok")
        # Concatenate embeddings
        vector = torch.cat([user_embedding, item_embedding, demographic_embedding, genre_embedding], dim=-1)
        print("concat ok")
        # Pass through fully connected layers
        for fc_layer in self.fc_layers:
            vector = fc_layer(vector)
            vector = F.relu(vector)

        # Output layer
        logits = self.affine_output(vector)
        rating = self.logistic(logits)
        return rating.squeeze()

    def init_weight(self):
        pass


In [22]:
#collapse-hide
parser = argparse.ArgumentParser()
parser.add_argument("--seed",
	type=int,
	default=42,
	help="Seed")
parser.add_argument("--lr",
	type=float,
	default=0.001,
	help="learning rate")
parser.add_argument("--dropout",
	type=float,
	default=0.2,
	help="dropout rate")
parser.add_argument("--batch_size",
	type=int,
	default=256,
	help="batch size for training")
parser.add_argument("--epochs",
	type=int,
	default=30,
	help="training epoches")
parser.add_argument("--top_k",
	type=int,
	default=10,
	help="compute metrics@top_k")
parser.add_argument("--factor_num",
	type=int,
	default=32,
	help="predictive factors numbers in the model")
parser.add_argument("--layers",
    nargs='+',
    default=[64,32,16,8],
    help="MLP layers. Note that the first layer is the concatenation of user \
    and item embeddings. So layers[0]/2 is the embedding size.")
parser.add_argument("--num_ng",
	type=int,
	default=4,
	help="Number of negative samples for training set")
parser.add_argument("--num_ng_test",
	type=int,
	default=100,
	help="Number of negative samples for test set")
parser.add_argument("--out",
	default=True,
	help="save model or not")

_StoreAction(option_strings=['--out'], dest='out', nargs=None, const=None, default=True, type=None, choices=None, required=False, help='save model or not', metavar=None)

In [210]:
# set device and parameters
args = parser.parse_args("")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# load data


# set the num_users, items, demographic features, and genre features
num_users = data_data['user_id'].nunique() + 1
num_items = data_data['item_id'].nunique() + 1
num_demographic_features = len(data_data.columns) - 4
num_demographic_features = 25 #these num of features are all fucked up
print("demo_feat: ", num_demographic_features)
num_genre_features = 50  #should be 18 tbh

# construct the train and test datasets
data = NCF_Data2(args, data_data)
train_loader = data.get_train_instance()

test_loader = data.get_test_instance()

# set model and loss, optimizer
model = MLP2(args, num_users, num_items, num_demographic_features, num_genre_features)
model = model.to(device)
loss_function = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=args.lr)

# train, evaluation
best_hr = 0
for epoch in range(1, args.epochs + 1):
    model.train()  # Enable dropout (if have).
    start_time = time.time()

    for user, item, demographic, genre, label in train_loader:
        user = user.to(device)
        item = item.to(device)
        demographic = demographic.to(device)
        genre = genre.to(device)
        label = label.to(device)
        #print(demographic)

        demographic = demographic.long()
        genre = genre.long()
        print("genre", genre)
        print("demo:", demographic)
        #print(demographic)
        #print(genre)


        optimizer.zero_grad()
        prediction = model(user, item, demographic, genre)
        loss = loss_function(prediction, label)
        loss.backward()
        optimizer.step()

    model.eval()
    HR, NDCG = metrics2(model, test_loader, args.top_k, device)

    elapsed_time = time.time() - start_time
    print("Epoch {:03d}".format(epoch) + " time to train: " +
          time.strftime("%H: %M: %S", time.gmtime(elapsed_time)))
    print("HR: {:.3f}\tNDCG: {:.3f}".format(np.mean(HR), np.mean(NDCG)))

    if HR > best_hr:
        best_hr = HR
        best_ndcg = NDCG
        best_epoch = epoch
        if args.out:
            if not os.path.exists(MODEL_PATH):
                os.mkdir(MODEL_PATH)
            torch.save(model,
                       '{}{}.pt'.format(MODEL_PATH, model.__class__.__name__))


demo_feat:  25


<ipython-input-207-f73f3d11ac08>:54: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  interact_status['negative_samples'] = interact_status['negative_items'].apply(lambda x: random.sample(x, self.num_ng_test))
<ipython-input-207-f73f3d11ac08>:60: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  train_ratings['negatives'] = train_ratings['negative_items'].apply(lambda x: random.sample(x, self.num_ng))
Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 239, in _feed
    reader_close()
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another except

['user_id', 'item_id', 'rating', 'age', 'gender_F', 'gender_M', 'occupation_administrator', 'occupation_artist', 'occupation_doctor', 'occupation_educator', 'occupation_engineer', 'occupation_entertainment', 'occupation_executive', 'occupation_healthcare', 'occupation_homemaker', 'occupation_lawyer', 'occupation_librarian', 'occupation_marketing', 'occupation_none', 'occupation_other', 'occupation_programmer', 'occupation_retired', 'occupation_salesman', 'occupation_scientist', 'occupation_student', 'occupation_technician', 'occupation_writer', 'unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'FilmNoir', 'Horror', 'Musical', 'Mystery', 'Romance', 'SciFi', 'Thriller', 'War', 'Western', 'rank_latest', 'negative_items', 'negatives']
ora rows:
[0, 0, 0, 1.0, 49, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, 0, 0, 0,

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:

genre tensor([ 0,  1,  0, 49,  0])
demo: tensor([0, 0, 0, 1, 0])
users ok
items ok
demo ok
inside genre tensor([ 0,  1,  0, 49,  0])
genres ok
concat ok


IndexError: Caught IndexError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "<ipython-input-203-ab449014a5bf>", line 18, in __getitem__
    genre = self.genre_list[idx]
IndexError: list index out of range


In [ ]:
model

GMF(
  (embedding_user): Embedding(944, 32)
  (embedding_item): Embedding(1683, 32)
  (affine_output): Linear(in_features=32, out_features=1, bias=True)
  (logistic): Sigmoid()
)

In [ ]:
ml_100k = pd.read_csv(
	DATA_PATH,
	sep="\t",
	names = ['user_id', 'item_id', 'rating', 'timestamp'],
	engine='python')

In [ ]:
ml_100k.describe()

,user_id,item_id,rating,timestamp
count,100000.00000,100000.000000,100000.000000,1.000000e+05
mean,462.48475,425.530130,3.529860,8.835289e+08
std,266.61442,330.798356,1.125674,5.343856e+06
min,1.00000,1.000000,1.000000,8.747247e+08
25%,254.00000,175.000000,3.000000,8.794487e+08
50%,447.00000,322.000000,4.000000,8.828269e+08
75%,682.00000,631.000000,4.000000,8.882600e+08
max,943.00000,1682.000000,5.000000,8.932866e+08


In [ ]:
# set device and parameters
args = parser.parse_args("")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# load data
ml_100k = pd.read_csv(
	DATA_PATH,
	sep="\t",
	names = ['user_id', 'item_id', 'rating', 'timestamp'],
	engine='python')

# set the num_users, items
num_users = ml_100k['user_id'].nunique()+1
num_items = ml_100k['item_id'].nunique()+1

# construct the train and test datasets
data = NCF_Data(args, ml_100k)
train_loader = data.get_train_instance()
test_loader = data.get_test_instance()

# set model and loss, optimizer
model = MLP(args, num_users, num_items)
model = model.to(device)
loss_function = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=args.lr)

# train, evaluation
best_hr = 0
for epoch in range(1, args.epochs+1):
	model.train() # Enable dropout (if have).
	start_time = time.time()

	for user, item, label in train_loader:
		user = user.to(device)
		item = item.to(device)
		label = label.to(device)

		optimizer.zero_grad()
		prediction = model(user, item)
		loss = loss_function(prediction, label)
		loss.backward()
		optimizer.step()
		#writer.add_scalar('loss/Train_loss', loss.item(), epoch)

	model.eval()
	HR, NDCG = metrics(model, test_loader, args.top_k, device)
	#writer.add_scalar('Perfomance/HR@10', HR, epoch)
	#writer.add_scalar('Perfomance/NDCG@10', NDCG, epoch)

	elapsed_time = time.time() - start_time
	print("Epoch {:03d}".format(epoch) + " time to train: " +
			time.strftime("%H: %M: %S", time.gmtime(elapsed_time)))
	print("HR: {:.3f}\tNDCG: {:.3f}".format(np.mean(HR), np.mean(NDCG)))

	if HR > best_hr:
		best_hr, best_ndcg, best_epoch = HR, NDCG, epoch
		if args.out:
			if not os.path.exists(MODEL_PATH):
				os.mkdir(MODEL_PATH)
			torch.save(model,
				'{}{}.pt'.format(MODEL_PATH, model.__class__.__name__))


<ipython-input-6-87c186f5da76>:52: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  interact_status['negative_samples'] = interact_status['negative_items'].apply(lambda x: random.sample(x, self.num_ng_test))
<ipython-input-6-87c186f5da76>:58: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  train_ratings['negatives'] = train_ratings['negative_items'].apply(lambda x: random.sample(x, self.num_ng))
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warnin

Epoch 001 time to train: 00: 00: 31
HR: 0.390	NDCG: 0.208
Epoch 002 time to train: 00: 00: 30
HR: 0.411	NDCG: 0.218
Epoch 003 time to train: 00: 00: 31
HR: 0.400	NDCG: 0.219
Epoch 004 time to train: 00: 00: 32
HR: 0.402	NDCG: 0.220
Epoch 005 time to train: 00: 00: 31
HR: 0.420	NDCG: 0.227
Epoch 006 time to train: 00: 00: 30
HR: 0.442	NDCG: 0.242
Epoch 007 time to train: 00: 00: 30
HR: 0.470	NDCG: 0.260
Epoch 008 time to train: 00: 00: 31
HR: 0.470	NDCG: 0.269
Epoch 009 time to train: 00: 00: 31
HR: 0.497	NDCG: 0.279
Epoch 010 time to train: 00: 00: 30
HR: 0.512	NDCG: 0.290
Epoch 011 time to train: 00: 00: 30
HR: 0.521	NDCG: 0.291
Epoch 012 time to train: 00: 00: 30
HR: 0.542	NDCG: 0.300
Epoch 013 time to train: 00: 00: 30
HR: 0.533	NDCG: 0.296
Epoch 014 time to train: 00: 00: 31
HR: 0.562	NDCG: 0.313
Epoch 015 time to train: 00: 00: 31
HR: 0.554	NDCG: 0.306
Epoch 016 time to train: 00: 00: 30
HR: 0.549	NDCG: 0.307
Epoch 017 time to train: 00: 00: 30
HR: 0.551	NDCG: 0.314
Epoch 018 time

In [ ]:
model

MLP(
  (embedding_user): Embedding(944, 32)
  (embedding_item): Embedding(1683, 32)
  (fc_layers): ModuleList(
    (0): Linear(in_features=64, out_features=32, bias=True)
    (1): Linear(in_features=32, out_features=16, bias=True)
    (2): Linear(in_features=16, out_features=8, bias=True)
  )
  (affine_output): Linear(in_features=8, out_features=1, bias=True)
  (logistic): Sigmoid()
)

## LAB CHALLENGE: Neural Matrix Factorization
So far we have developed GMF that applies a linear kernel to model the latent feature interactions and MLP that uses a non-linear kernel to learn the interaction function from data.

How can we fuse GMF and MLP under the NCF framework, so that they can mutually reinforce each other to better model the complex user-item interactions?

A straightforward solution is to let GMF and MLP share the same embedding layer and then combine the outputs of their interaction functions.

To provide flexibility to the fused model we want to allow GMF and MLP to learn separate embeddings and combine two models by concatenating their last hidden layer.

Formally

$$\phi^{GMF} = \mathbf{p_u}^G\odot \mathbf{q_i}^G$$
$$\phi^{MLP} = a_L(\mathbf{W}_L^T(a_{L-1}(...a_2 (\mathbf{W}_2^T \begin{bmatrix}
\mathbf{p_u} \\ \mathbf{q_i}
\end{bmatrix} + \mathbf{b}_2)...)) + \mathbf{b}_L)$$

$$ y_{ui} = \sigma(\mathbf{h}^T \begin{bmatrix}
\ \phi^{GMF} \\ \phi^{MLP}
\end{bmatrix})$$

that results in the following model architecture

<center>  <img src="https://drive.google.com/uc?export=view&id=1gNLUpiQdbDPMdvfZYVs3lcou3cd4Favb" width="550" height="400"> </center>


- TASK 1: implement the model as described
- TASK 2: compare the performance of such model with the GMS and MLP models using the metrics provided.
- TASK 3: tune the networks by plotting HR@10 and NDCG@10 with respect to the number of predictive factors [8, 16, 32, 64] for all the 3 algorithms
